In [1]:
import os
import subprocess
import re
import math
import pandas as pd
import numpy as np

from glass.rd.shp import shp_to_obj
from glass.wt.shp import df_to_shp

from glass.wenv.grs import run_grass

from glass.prop.prj imporshp_epsg
from glass.pys.oss import fprop


In [2]:
#dados IRv4
IRv4_shp='/home/alvaro/DGT/dados/treino/buffer_prodfacility_CBR.shp'
#dados dgt
cae_shp='/home/alvaro/DGT/dados/treino/AreasEdificadas2018_CBR.shp'

# IRv4/CAE intersection result
IRv4vscae='/home/alvaro/DGT/dados/treino/Irv4vscae.shp'

#IRv4 com distancia aos dados cae
IRv4_dist_cae='/home/alvaro/DGT/dados/treino/Irv4_dist_cae.shp'

# final results
IRv4_existe_cae ='/home/alvaro/DGT/dados/treino/AE_IRD.shp'
IRv4_no_cae='/home/alvaro/DGT/dados/treino/AE_IRdnCAE.shp'

In [3]:
# Start GRASS GIS Session

# Define Reference Raster, in this case the ndci
refshp= '/home/alvaro/DGT/lim_adminis/PT/Lim_PT.shp'
outfolder='/home/alvaro/DGT/'

epsg = shp_epsgshp)

loc = 'pext'

# Start GRASS GIS Session
gb = run_grass(
    outfolder, grassBIN='grass78', location=loc,
    srs=epsg
)
import grass.script.setup as gsetup
from grass.pygrass.modules import Module  # import grass modules

# __flag = 'o' if not lmtExt else 'or'

gsetup.init(gb, outfolder, loc, 'PERMANENT')

'/tmp/tmpipwkerum'

In [4]:
# GRASS GIS Modules

from glass.it.shp import shp_to_grs, grs_to_shp
from glass.defs_grass import overlay_grs, dissolve_grs, add_column_grs, distance_grs

In [5]:
# Import data

IRv4_grs=shp_to_grs(IRv4_shp, fprop(IRv4_shp, 'fn'))
cae_grs=shp_to_grs(cae_shp, fprop(cae_shp, 'fn'))

         overwritten
         overwritten


In [6]:
# Union Irv4 CAE
IRv4cae=overlay_grs(IRv4_grs, cae_grs, 'or', 'union_Irv4cae')

In [7]:
#add coluna de atributos

dist={'dist_cae':'DOUBLE PRECISION'}
newIRv4_grs=add_column_grs(IRv4_grs, dist)

In [8]:
# distance Irv4 CAE
dist_IRv4cae=distance_grs(newIRv4_grs, cae_grs, 'dist', 'dist_cae')

In [9]:
# Export Data
IRv4distcae=grs_to_shp(dist_IRv4cae, IRv4_dist_cae, 'area')

IRv4_and_cae=grs_to_shp(IRv4cae, IRv4vscae, 'area')

In [10]:
##read a shapefile and convert to geodataframe 

gdf=shp_to_obj(IRv4_and_cae)

IRv4df=shp_to_obj(IRv4distcae)

In [11]:
gdf = gdf[~gdf.a_cat.isna()]

gdf['a_cat'] = gdf.a_cat.astype(int)
gdf['b_cat'] = gdf.b_cat.fillna(value=0)
gdf['b_cat'] = gdf.b_cat.astype(int)

In [12]:
# Count how many times we have the same a_cat

catcount = pd.DataFrame({
    'countcat' : gdf.groupby(['a_cat'])['a_cat'].agg('count')
}).reset_index()

catcount.rename(columns={'a_cat': 'acaty'}, inplace=True)

In [13]:
# Join
gdf = gdf.merge(catcount, how='inner', left_on='a_cat', right_on='acaty')

In [14]:
# Get classes
gdf['fcat'] = -1

gdf['fcat'] = np.where(
    (gdf.countcat == 1) & (gdf.b_cat == 0),
    0, gdf.fcat
)

gdf['fcat'] = np.where(
    (gdf.countcat == 1) & (gdf.b_cat > 0),
    1, gdf.fcat
)

gdf['fcat'] = np.where(
    gdf.countcat > 1, 
    2, gdf.fcat
)


In [15]:
fdf = pd.DataFrame({
    'existcae' : gdf.groupby(["a_cat"])['fcat'].agg('min')
}).reset_index()

In [16]:
IRv4df['cat'] = IRv4df.index + 1

#junçao dos dados do IRv4 com os dados do grau de existencia
IRv4df = IRv4df.merge(fdf, how='left', left_on='cat', right_on='a_cat')


In [17]:

IRv4df.rename(columns={
    'cat' : 'id_obj', 'cat_' : 'id_IRv4'
}, inplace=True)

IRv4df['fonte']    = 'IRv4'
IRv4df['classuos'] = '1211'
IRv4df['dist_cae'] = IRv4df.dist_cae.fillna(value=0)
IRv4df['existcae'] = IRv4df.existcae.fillna(value=-1)
IRv4df['existcae'] = IRv4df.existcae.astype(int)
IRv4df['areaha']   = IRv4df.geometry.area / 10000

IRv4df.drop(['a_cat'], axis=1, inplace=True)


In [18]:
#seleção de atributos do conjunto de dados do IRv4df
resultado_1=IRv4df[['id_obj','id_IRv4','geometry', 'fonte', 'classuos', 'existcae', 'areaha']]

In [20]:
#export to shapefille
df_to_shp(resultado_1, IRv4_existe_cae)

'/home/alvaro/DGT/dados/treino/AE_IRD.shp'

In [21]:
# seleção de das areas nao incluidas na CAE

no_cae=IRv4df[(IRv4df["existcae"]==0)]
no_cae=no_cae.reset_index()

no_cae['id_n_cae'] = no_cae.index + 1


In [22]:
resultado_2=no_cae[['id_n_cae', 'id_obj','id_IRv4','geometry', 'fonte', 'classuos', 'areaha', 'dist_cae']]

In [24]:
#export to shapefille
df_to_shp(resultado_2, IRv4_no_cae)

'/home/alvaro/DGT/dados/treino/AE_IRdnCAE.shp'